In [1]:
# import modules
from qutip import *
import numpy as np
from math import log2
from scipy.linalg import logm, expm, inv
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import reduce
from itertools import product
from IPython.display import display, Latex
import importlib
import pickle
import time
import random
import os

# define identity for qubits 
I2 = qeye(2)

# define Pauli matrices and constants
σ_x = sigmax()
σ_y = sigmay()
σ_z = sigmaz()
π = np.pi

# font specs for plots
# %matplotlib inline
mpl.rcParams["font.family"] = "STIXGeneral"
mpl.rcParams["mathtext.fontset"] = "stix"
mpl.rcParams["font.size"] = "14"

In [2]:
def quantities(N, Jx, Jy, Jz, periodic_bc, tolerance):

    
    if N % 2 != 0: 
        raise ValueError("Please enter an even number of sites")
    
    # define Pauli matrices and constants
    σ_x = sigmax()
    σ_y = sigmay()
    σ_z = sigmaz()
    π = np.pi

    # Interaction coefficients, which we assume are uniform throughout the lattice
    Jx_list = Jx*np.ones(N)
    Jy_list = Jy*np.ones(N)
    Jz_list = Jz*np.ones(N)

    # Setup operators for individual qubits; 
    # here sx_list[j] = X_j, sy_list[j] = Y_j, and sz_list[j] = Z_j
    # since the Pauli matrix occupies the jth location in the tensor product of N terms
    # of which N-1 terms are the identity
    sx_list, sy_list, sz_list = [], [], []

    for i in range(N):
        op_list = [qeye(2)]*N
        op_list[i] = σ_x
        sx_list.append(tensor(op_list))
        op_list[i] = σ_y
        sy_list.append(tensor(op_list))
        op_list[i] = σ_z
        sz_list.append(tensor(op_list))

    # define variable for total Hamiltonian H_N and the list of all local 
    # Hamiltonians H_list
    H_N = 0 
    H_list = []
    
    # collect 
    for j in range(N - 1):

        # find H_ij, the Hamiltonian between the ith and jth sites 
        H_ij = Jx_list[j] * sx_list[j] * sx_list[j + 1] + \
               Jy_list[j] * sy_list[j] * sy_list[j + 1] + \
               Jz_list[j] * sz_list[j] * sz_list[j + 1]
        
        # add H_ij to H_N and append H_ij to H_list
        H_N += H_ij
        H_list.append(H_ij)

    # execute if periodic boundary conditions are specified
    if periodic_bc: 
        
        # find H_N1, the Hamiltonian between the Nth and first site
        H_N1 = Jx_list[N-1] * sx_list[N - 1] * sx_list[0] + \
               Jy_list[N-1] * sy_list[N - 1] * sy_list[0] + \
               Jz_list[N-1] * sz_list[N - 1] * sz_list[0]

        # add H_N1 to H_N and append H_N1 to H_list
        H_N += H_N1
        H_list.append(H_N1)

    # find eigenavlues and eigenstates of Hamiltonian 
    eigenvalues, eigenstates = H_N.eigenstates()

    # find indices of smallest eigenvalues, which correspond to the energy(ies) 
    # of the ground state (space in the case of degeneracy); 
    min_eigenvalue = min(eigenvalues)
    indices = [index for index, value in enumerate(eigenvalues) if np.allclose(value, min_eigenvalue, tolerance)]

    # find eigenstates corresponding to ground state
    eigenstates_list = eigenstates[indices]

    # create sum of density matrices of ground states in case ground state is degenerate
    ρ_ground_state = 0 
    for j in range(len(eigenstates_list)):
        ρ_ground_state += (eigenstates_list[j])*(eigenstates_list[j]).dag()

    # return normalized ground state
    return H_N, H_list, eigenstates, eigenvalues, min_eigenvalue, ρ_ground_state
def orthonormal_eigenstates(eigenstates, tolerance):
    # find the inner product between all eigenstates; 
    # note that if eigenstates are orthonormal this should form 
    # the identity matrix with dimension equal to number of eigenstates
    fidelity_list = []
    for i, eigenstate in enumerate(eigenstates):
        row = []
        for j in range(len(eigenstates)):
            row.append(fidelity(eigenstate, eigenstates[j]))
        fidelity_list.append(np.array(row))


    # round entry in fidelity_matrix according to value of tolerance
    # very whether this matrix if equal to the identity
    if np.array_equal(np.round(fidelity_list, decimals= -int(np.log10(tolerance))), np.eye(len(eigenstates))): 
        print(f'When N = {round(log2(len(eigenstates)))} your eigenstates form an orthonormal basis!')
    else: 
        print(f'When N = {round(log2(len(eigenstates)))} your eigenstates are not orthonormal')

In [3]:
# specify parameters of 2-site spin chain
N2 = 2
Jx = 1
Jy = 1
Jz = 1
periodic_bc = False
tolerance = 1e-12

# calculate spin chain quantities
# calculate spin chain quantities
H2, H2_list, eigenstates2, eigenvalues2, min_eigenvalue2, ρ_ground_state2 = \
    quantities(N2, Jx, Jy, Jz, periodic_bc, tolerance)

In [4]:
# define singlet and triplet states
S = eigenstates2[0]
T0 = eigenstates2[3]
Tn1 = eigenstates2[2]
T1 = eigenstates2[1]

# create basis of singlets and triplets
ST_basis = [S, Tn1, T0, T1]

# create matrix with S, Tn1, T0 and T1 as column vectors 
ST_matrix = Qobj(np.hstack([eigenstate.full() for eigenstate in ST_basis]))
display(ST_matrix)

# eigenvalues for 2-site spin chain
display(eigenvalues2)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.          0.          0.          1.        ]
 [-0.70710678  0.          0.70710678  0.        ]
 [ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          1.          0.          0.        ]]

array([-3.,  1.,  1.,  1.])

In [5]:
# specify parameters of spin chain 
N4 = 4
Jx = 1
Jy = 1
Jz = 1
periodic_bc = True
tolerance4 = 1e-12

# calculate spin chain quantities
H4, H4_list, eigenstates4, eigenvalues4, E_0, ρ_ground_state4 = \
    quantities(N4, Jx, Jy, Jz, periodic_bc, tolerance)

# verify that eigenstates are orthonormal 
orthonormal_eigenstates(eigenstates4, tolerance4)

When N = 4 your eigenstates form an orthonormal basis!


# $\text{GSP}_{\text{exact}} $

In [9]:
N_cavities = 4
N_qubits = 4
m_start = 1
m_stop = 4
α_start = 0
α_end = 1
α_steps = 100
num_states = 2
cutoff = 9
save_file = False

initial_state_list = spin_chain.generate_initial_states(N_cavities, N_qubits, num_states, cutoff)
GSP_exact_fidelity_array = spin_chain.plot_fidelity(N_cavities, N_qubits, eigenvalues4, eigenstates4,\
                                                    m_start, m_stop, α_start, α_end, α_steps, \
                                                    initial_state_list, cutoff, save_file)

NameError: name 'spin_chain' is not defined

In [17]:
# define how many cavities to project onto vacuum 
m_start = 4
m_stop = 4
num_states = 4

Ψ0_list = spin_chain.generate_initial_states(N_cavities, N_qubits, num_states, cutoff)
GSP_exact_fidelity_array = spin_chain.plot_fidelity(N_cavities, N_qubits, eigenvalues4, eigenstates4,\
                                               m_start, m_stop, α_start, α_end, α_steps, \
                                               Ψ0_list, cutoff, save_file)

# trace out cavities from initial state list so that we have a 
# list of products state of qubits unentangled with cavities
P14 = tensor([basis(cutoff, 0).dag()]*(N_cavities) + [qeye(2)]*(N_qubits))
ψ0_list = [P14*state for state in Ψ0_list] 

NameError: name 'spin_chain' is not defined

# $\text{GSP using Spectral Decomposition} $

In [6]:
def eigendecomp_spin_operators(N, ST_basis):
    
    '''
    function to find the global eigenkets for blue and red bonds as well
    as the spectrum of each local 2-site Hamiltonian in the basis of these
    global eigenkets
    '''
    
    # collect indices for H_ik Hamiltonians which will be used to convert 
    # blue global eigenkets to red global eigenkets
    H_ik_indices = [(2 + i, N - i) for i in range(0, int(N/2) - 1)]

    # collect all H_ik matrices, which are Hamiltonians containing 
    # XX + YY + ZZ at indices i and k specified by each tuple in H_SWAP_indices
    H_ik_list = []
    for indices_tuple in H_ik_indices:

        # collect at which sites we will insert Pauli matrices
        site1 = indices_tuple[0]
        site2 = indices_tuple[1]

        # define list of identity operators and insert Pauli matrices
        # at sites specified by indices_tuple
        op_list = [qeye(2)]*N
        op_list[site1 - 1] = σ_x
        op_list[site2 - 1] = σ_x
        HX_couplings = tensor(op_list)
        op_list[site1 - 1] = σ_y
        op_list[site2 - 1] = σ_y
        HY_couplings = tensor(op_list)
        op_list[site1 - 1] = σ_z
        op_list[site2 - 1] = σ_z
        HZ_couplings = tensor(op_list) 

        # append H_SWAP = HX_couplings + HY_couplings + HZ_couplings to H_ik_list
        H_ik_list.append(HX_couplings + HY_couplings + HZ_couplings)

    # collect product of SWAP matrices that will convert red eigenkets to blue eigenkets
    I = tensor([qeye(2)]*N)
    dim = 2**N
    SWAP_operator = Qobj(reduce(lambda x,y: x*y, \
                                [(H_ik + I)/2 for H_ik in H_ik_list]), dims = [[dim], [dim]])

    # construct global eigenkets for red and blue bonds for N sites
    blue_mpsN = [Qobj(eigvec) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]
    red_mpsN = [SWAP_operator(Qobj(eigvec)) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]
    
    # initialize empty list which will contain the spectrum of each local Hamiltonian 
    # in the basis of the blue and red global eigenkets
    H_local_spectrum_list = []

    # find spectra of all 2-site Hamiltonians
    for i in range(1, N+1, 2):

        # compute spectrum from tensor products of index 1 to i 
        Hij_spectrum_1i = np.tile(eigenvalues2, 2**(i-1))

        # compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
        Hij_spectrum_1N = np.repeat(Hij_spectrum_1i, np.ceil(2**(N - i - 1)))

        # append to Hij_spectrum_list
        H_local_spectrum_list.append(Hij_spectrum_1N)

    # extra spectra for blue and red bonds according to previous ordering of eigenkets
    blue_spectraN = H_local_spectrum_list
    red_spectraN = H_local_spectrum_list[::-1]

    return blue_spectraN, blue_mpsN, red_spectraN, red_mpsN
def reconstruct_HN_list(HN_list, blue_spectraN, blue_mpsN, red_spectraN, red_mpsN):

    '''
    reconstruct list of local Hamiltonians from parameters yielded by the eigendecomposition 
    of all red and blue bonds
    '''

    # calculate the number of sites in spin chain
    N = 2*len(blue_spectraN)

    # reconstruct lists of Hamiltonians corresponding to blue and red bonds
    HN_blue_list_reconstructed = [sum([λ*vec*vec.dag() for λ, vec in \
                                  zip(blue_spectraN[i], blue_mpsN)]) for i in range(int(N/2))]
    HN_red_list_reconstructed = [sum([λ*vec*vec.dag() for λ, vec in \
                                 zip(red_spectraN[i], red_mpsN)]) for i in range(int(N/2))]

    # interleave HN_blue_list and HN_red_list to create an ordered list of local Hamiltonians 
    HN_list_reconstructed = np.concatenate((HN_blue_list_reconstructed, HN_red_list_reconstructed))
    HN_list_reconstructed[::2] = HN_blue_list_reconstructed
    HN_list_reconstructed[1::2] = HN_red_list_reconstructed 

    # extract lists of blue and red Hamiltonians from HN_list
    HN_blue_list = HN_list[::2]
    HN_red_list = HN_list[1::2]

    # verify that HN_list == HN_list_reconstructed using two methods
    verification1 = np.allclose(HN_list, HN_list_reconstructed, atol = 1e-12)

    # use for loop to check whether HN_list[i] == HN_list_reconstructed[i]
    HN_tot_bool = []
    for Hb1, Hb2, Hr1, Hr2 in zip(HN_blue_list_reconstructed, HN_blue_list, \
                                  HN_red_list_reconstructed, HN_red_list):

        HN_tot_bool.append(np.allclose(Hb1, Hb2, atol = 1e-12))
        HN_tot_bool.append(np.allclose(Hr1, Hr2, atol = 1e-12))

    verification2 = all(HN_tot_bool)

    # print whether both methods of verifying equality between Hamiltonians 
    # yield True or False
    if verification1 and verification2: 
        print('Hamiltonians are the same!')
    else: 
        print('Hamiltonians are different')   
def compute_VTA_list_GWT(α_start, α_end, α_steps):
    
    '''
    list of VTAs using the Gaussian Weight Tensor approach
    '''
    
    # start the timer for the operation 
    start_time = time.time()
    
    # define array over which we will sweep α
    α_array = np.linspace(α_start, α_end, α_steps)
    
    # calculate the vacuum transition amplitude for N = 4
    VTA_list = []
    for α in α_array: 
        VTA = sum(
                W4(α, k0, k3, k2, k1)*W4(α, k0, k1, k2, k3) * \
                W4(α, k2, k1, k0, k3)*W4(α, k2, k3, k0, k1) * \
                (ν4_bras[k3]*μ4_kets[k2])*(μ4_bras[k2]*ν4_kets[k1]) * \
                (ν4_bras[k1]*μ4_kets[k0])*(ν4_kets[k3]*μ4_bras[k0])  
                for k3 in range(len(d)) \
                for k2 in range(len(c)) \
                for k1 in range(len(b)) \
                for k0 in range(len(a)) \
                )
        # adjust dimensions of VTA and append to VTA_list
        VTA_adjusted = Qobj(VTA, dims = [[2]*4, [2]*4])
        VTA_list.append(VTA_adjusted)
        
    # Saving the list of quantum objects to a file
    path = os.getcwd() + f'/four_site_data/VTA{N}_GWT_{α_start}_{α_end}_{α_steps}.pkl'

    # save VTA_list to directory specified by path 
    with open(path, 'wb') as file:
        pickle.dump(VTA_list, file)
        
    # to upload saved data use following code
#     with open(path, 'rb') as file:
#         VTA_list_new = pickle.load(file)
        
    print("%s seconds" % (time.time() - start_time))
    return VTA_list
def VTA_trotter(VTA, ψ, r): 
    for _ in range(r):
        ψ = (VTA*ψ).unit()
    return ψ
def plot_GSP_data(ψ0_list, VTA_list, α_start, α_end, α_steps, r, 
                  m_start, m_stop, fidelity_thresholds, save_file): 
    
    # define array over which we will plot α
    α_array = np.linspace(α_start, α_end, α_steps)

    # define list of colors to use for plots 
    colors = ['blue', 'orange', 'green','red', 'purple', 'black', 'pink', 'gray', 'cyan', 'magenta']

    # compute list of projection fidelity for VTA_list for all initial states in ψ0_list
    GSP_fidelity_array_list = [[expect(ρ_ground_state4, VTA_trotter(VTA, ψ0, r))  \
                                for VTA in VTA_list] \
                                for ψ0 in ψ0_list]

    # generate custom linestyles to use if we are projecting 4 or more cavities onto vacuum
    dashed_list = [(None, None), (5, 2), (1, 2), (5, 2, 1, 2), (10, 5, 2, 5), \
                   (2, 2, 10, 2), (5, 2, 10, 2, 5, 2), (15, 5, 5, 5), (2, 5, 10, 5)]

    fig, ax = plt.subplots(figsize=(8, 6))

    if fidelity_thresholds:

        # GSP operator for large α
        G = π41[0]*π23[0]*π34[1]*π12[1]*π41[1]*π23[1]*π34[1]*π12[1] + \
            π41[1]*π23[1]*π34[0]*π12[0]*π41[1]*π23[1]*π34[1]*π12[1] + \
            π41[1]*π23[1]*π34[1]*π12[1]*π41[0]*π23[0]*π34[1]*π12[1] + \
            π41[1]*π23[1]*π34[1]*π12[1]*π41[1]*π23[1]*π34[0]*π12[0] 

        # compute thresholds at which the fidelity will saturate
        threshold_array_list = [[expect(ρ_ground_state4, (G*ψ0).unit())]*(α_steps) \
                                 for ψ0 in ψ0_list]

        # graph the theoretical projection of all initial states onto the true ground state 
        for i, (GSP_list, threshold_list) in enumerate(zip(GSP_fidelity_array_list, \
                                                           threshold_array_list)): 
            ax.plot(α_array, GSP_list, \
                    label=f'$ | \psi_{i} \\rangle, F_0$ = {GSP_list[0] : .3f}', \
                    color=colors[i], \
                    linestyle='-')
            ax.plot(α_array, threshold_list, \
                    label = f'$\mathcal{{G}}_{i} = {threshold_list[i]:.3f}$', \
                    color = colors[i], \
                    linestyle = '--')

    else: 
        # graph the theoretical projection of all initial states onto the true ground state 
        for i, fidelity_array in enumerate(GSP_fidelity_array_list): 
            ax.plot(α_array, fidelity_array, \
                    label=f'M = 4, $F_0$ = {fidelity_array[0] : .3f}', \
                    color=colors[i], \
                    linestyle='-')

    ax.set_xlabel(r'$\alpha$', fontsize=15)
    ax.set_ylabel('Fidelity', fontsize=15)
    ax.set_title(f'GSP with r = {r} for N = 4', fontsize=17.5)  
    ax.legend(fontsize = 12.5)
    ax.grid(True)

    if save_file: 

        # define which folder subfolder in /data to store the image
        if N_qubits == 2: 
            site_folder = 'two_sites'
        elif N_qubits == 3: 
            site_folder = 'three_sites'
        elif N_qubits == 4: 
            site_folder = 'four_sites'
        elif N_qubits == 5: 
            site_folder = 'five_sites'

        # calculate number of states that were simulated
        num_states = len(ψ0_list)

        if m_start != m_stop: 

            # create name for png file 
            filename = f'/Users/lukebell/Documents/boson_gang/data/{site_folder}/simulated_' \
                        f's{num_states}_m{m_start}{m_stop}_r{r}.png'

            # Check if the file already exists
            if os.path.exists(filename):
                # If it exists, remove it
                os.remove(filename)

            # save figure 
            plt.savefig(filename)

        elif m_start == m_stop: 

            # create name for png file 
            filename = f'/Users/lukebell/Documents/boson_gang/data/{site_folder}/simulated_' \
                        f's{num_states}_m{m_start}_r{r}.png'

        # Check if the file already exists
        if os.path.exists(filename):
            # If it exists, remove it
            os.remove(filename) 

        # save figure 
        plt.savefig(filename)

In [7]:
# verify eigendecomposition can reconstruct list of local Hamiltonians for N = 4
N = 4
blue_spectra4, blue_mps4, red_spectra4, red_mps4 = eigendecomp_spin_operators(N, ST_basis)
reconstruct_HN_list(H4_list, blue_spectra4, blue_mps4, red_spectra4, red_mps4)

Hamiltonians are the same!


In [10]:
# define local Hamiltonians 
A = H4_list[0]
B = H4_list[1]
C = H4_list[2]
D = H4_list[3]

# collect an ordered list of all the spectra of the local Hamiltonians
spectra4 = blue_spectra4 + red_spectra4
spectra4[::2] = blue_spectra4
spectra4[1::2] = red_spectra4

# define lists of eigenvalues for spin operators A, B, C and D that incorporate the 
# translation - E_0/4 
a = spectra4[0] - E_0/4
b = spectra4[1] - E_0/4 
c = spectra4[2] - E_0/4
d = spectra4[3] - E_0/4

# define Gaussian weight tensor
W4 = lambda α, i, j, k, l: np.exp(-((α**2)/2)*(a[i] + b[j] + c[k] + d[l])**2)

# define all the bras and kets of the blue and red global eigenstates
μ4_kets = blue_mps4
μ4_bras = [eigvec.dag() for eigvec in μ4_kets]
ν4_kets = red_mps4
ν4_bras = [eigvec.dag() for eigvec in ν4_kets]

# upload list VTA operators l
path = os.getcwd() + f'/four_site_data/VTA{N}_GWT_{α_start}_{α_end}_{α_steps}.pkl'
with open(path, 'rb') as file:
         VTA_list_GWT = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lukebell/Documents/boson_gang/4_sites/four_site_data/VTA4_GWT_0_1_100.pkl'

In [ ]:
r = 1
fidelity_tresholds = True
save_file = False

plot_GSP_data(ψ0_list, VTA_list_GWT, α_start, α_end, α_steps, r, \
              m_start, m_stop, fidelity_tresholds, save_file)    

# $\text{GSP using  SU(2) Exponential Formula} $

In [11]:
def generate_SWAP_operators(N):

    
    if N % 2 != 0: 
        raise ValueError("Please enter an even number of sites.")
        
    # define zero and identity matrices corresponding to dimensions of VTA
    zeros_N = qzero([2]*N)
    I_N = qeye([2]*N)
    
    # define Pauli matrices and constants
    σ_x = sigmax()
    σ_y = sigmay()
    σ_z = sigmaz()

    # Setup operators for individual qubits; 
    # here σ_x_list[j] = X_j, σ_y_list[j] = Y_j, and σ_z_list[j] = Z_j
    # since the Pauli matrix occupies the jth location in the tensor product of N terms
    # for which (N-1) terms are the identity
    σ_x_list, σ_y_list, σ_z_list = [], [], []

    for i in range(N):
        op_list = [qeye(2)]*N
        op_list[i] = σ_x
        σ_x_list.append(tensor(op_list))
        op_list[i] = σ_y
        σ_y_list.append(tensor(op_list))
        op_list[i] = σ_z
        σ_z_list.append(tensor(op_list))

    # define empty lists for + and - projection operators
    π_list = []
    
    # collect list of all tuples corresponding to π_p and π_m 
    # SWAP operators
    for k in range(N):

        # find H_ij, the Hamiltonian between the ith and jth sites 
        H_kl = σ_x_list[k] * σ_x_list[(k + 1) % N] + \
               σ_y_list[k] * σ_y_list[(k + 1) % N] + \
               σ_z_list[k] * σ_z_list[(k + 1) % N]
        
        # add π_p to π_m to π_p_list and π_m_list, respectively
        π_p = (3 + H_kl)/4
        π_m = (1 - H_kl)/4
        π_list.append((π_p, π_m))
    
    # check to ensure projectors obey established summation and orthogonality relations
    π_kl_bool_list = []
    for π_kl in π_list: 
        π_kl_bool_list.append(π_kl[0] * π_kl[1] == zeros_N and \
                              π_kl[0] + π_kl[1] == I_N)

    if all(π_kl_bool_list):
        display(Latex(r'$ \pi^{+}_{kl} \pi^{-}_{kl} = 0 \text{ and } $'
                      r'$\pi^{+}_{kl} + \pi^{-}_{kl} = \mathbb{1}$' 
                     rf'$ \ \forall \ k,l \in \{{1, \dots, {N} \}}$'))
    else: 
        display(Latex(r'$ \pi^{+}_{kl} \pi^{-}_{kl} \neq 0 \text{ of } $'
                      r'$\pi^{+}_{kl} + \pi^{-}_{kl} \neq \mathbb{1}$' 
                     rf'$ \ \forall \ k,l \in \{{1, \dots, {N} \}}$'))
        
    return π_list
def compute_VTA_list_SU2_manual(N, α_start, α_end, α_steps, E_0): 


    # define projection operators where π_kl is a tuple 
    # such that π_kl[0] = π^{+}_{kl} and π_kl[1] = π^{-}_{kl}
    π_12 = π_list[0]
    π_23 = π_list[1]
    π_34 = π_list[2]
    π_41 = π_list[3]

    # define array over which we will sweep α and 
    # define r
    α_array = np.linspace(α_start, α_end, α_steps)
    r = 2 + E_0/2

    # define 0 matrix corresponding to dimensions of VTA matrix
    zeros_N = qzero([2]*N)

    # compute VTA for each value of α in α_array
    VTA_list_SU2 = []
    for α in α_array:
        VTA4 = zeros_N
        for b7 in range(2):
            for b6 in range(2):
                for b5 in range(2):
                    for b4 in range(2):
                        for b3 in range(2):
                            for b2 in range(2):
                                for b1 in range(2):
                                    for b0 in range(2):
                                        VTA4 += np.exp((-2*α**2) * (
                                                    (r - 4 + 2*(b0 + b3 + b5 + b6))**2 + \
                                                    (r - 4 + 2*(b0 + b2 + b5 + b7))**2 + \
                                                    (r - 4 + 2*(b1 + b2 + b4 + b7))**2 + \
                                                    (r - 4 + 2*(b1 + b3 + b4 + b6))**2)) * \
                                                     π_41[b7]*π_23[b6]*π_34[b5]*π_12[b4] * \
                                                     π_41[b3]*π_23[b2]*π_34[b1]*π_12[b0]
        VTA_list_SU2.append(VTA4)       
    return VTA_list_SU2
def MPO(α, b7, b6, b5, b4, b3, b2, b1, b0):
    
    # define projection operators where π_kl is a tuple 
    # such that π_kl[0] = π^{+}_{kl} and π_kl[1] = π^{-}_{kl}
    π_12 = π_list[0]
    π_23 = π_list[1]
    π_34 = π_list[2]
    π_41 = π_list[3]

    # define constant r
    r = 2 + E_0/2
    
    # return matrix product operator for a given α and set of indices
    return np.exp((-2*α**2) * (
           (r - 4 + 2*(b0 + b3 + b5 + b6))**2 + \
           (r - 4 + 2*(b0 + b2 + b5 + b7))**2 + \
           (r - 4 + 2*(b1 + b2 + b4 + b7))**2 + \
           (r - 4 + 2*(b1 + b3 + b4 + b6))**2)) * \
            π_41[b7]*π_23[b6]*π_34[b5]*π_12[b4] * \
            π_41[b3]*π_23[b2]*π_34[b1]*π_12[b0]
def compute_VTA_list_SU2_automated(N, α_start, α_end, α_steps): 
    
    # define array over which we will sweep α
    α_array = np.linspace(α_start, α_end, α_steps)

    # generate all possible combinations of tuples 
    combination_tuples = list(product([0, 1], repeat=int(N*(N/2))))
    
    # return the sum of all VTA
    return [sum(MPO(α, *combination) for combination in combination_tuples) \
               for α in α_array]

In [12]:
# generate list of projectors for N sites
N = 4
π_list = generate_SWAP_operators(N)

# define projection operators
π12 = π_list[0]
π23 = π_list[1]
π34 = π_list[2]
π41 = π_list[3]

<IPython.core.display.Latex object>

In [14]:
# compute list of VTAs using two methods -- one manually defining and another 
# automating the eight for loops
VTA_list_SU2_manual = compute_VTA_list_SU2_manual(N, α_start, α_end, α_steps, E_0)
VTA_list_SU2_automated = compute_VTA_list_SU2_automated(N, α_start, α_end, α_steps)

# verify the two lists are the same 
VTA_list_SU2_manual == VTA_list_SU2_automated

True

In [15]:
r = 1
fidelity_tresholds = True
save_file = False

plot_GSP_data(ψ0_list, VTA_list_GWT, α_start, α_end, α_steps, r, \
              m_start, m_stop, fidelity_tresholds, save_file)

NameError: name 'ψ0_list' is not defined

In [ ]:
np.allclose(VTA_list_GWT, VTA_list_SU2_automated)

# $\text{GSP}(\alpha, \lambda , E_0) = e^{ -\frac{\alpha^2 H_{\text{eff}}^2}{2}}) $

In [16]:
# define index for VTA_list and extra GSP and α
# corresponding to this index
indx = 23

# compute α and GSP correspondings to index
α_array = np.linspace(α_start, α_end, α_steps)
α = α_array[indx] 
VTA_list = VTA_list_SU2_automated
VTA = VTA_list[indx]
ψ0 = ψ0_list[0]

NameError: name 'ψ0_list' is not defined

In [ ]:
# np.allclose(GSP, Qobj(expm(logm(GSP))), atol = 1e-15)
H_eff = Qobj(logm(VTA)*(-2/(α**2)))
eigvals4_eff, eigvecs4_eff = H_eff.eigenstates()
np.allclose(VTA, Qobj(expm((-(α**2)/2)*H_eff)))

In [ ]:
max_GSP_eigvals4 = [max(VTA.eigenenergies()) for VTA in VTA_list]

threshold = min(max_GSP_eigvals4).real

if threshold.real == threshold:
    eigvals4_threshold = np.repeat(threshold, len(α_array))
else: 
    raise ValueError("Complex Eigenvalue Threshold")

fig, ax = plt.subplots(figsize=(8, 6))


ax.plot(α_array, max_GSP_eigvals4, color = 'b', label = 'Largest Eigenvalue')
ax.plot(α_array, eigvals4_threshold, color = 'r', linestyle = '--', \
        label = f'Threshold = {threshold : .3f}')
ax.set_xlabel(r'$\alpha$', fontsize=15)
ax.set_ylabel('Fidelity', fontsize=15)
ax.set_title(r' max{Sp(VTA($\alpha)$)} for N = 4', fontsize=17.5)  
ax.legend()
ax.grid(True)

In [ ]:
# once you renormalize, you can analytically show that the amplitude associated with the ground state 
# will approach 1, so this is it!
# it should be something like two iterations 

# Floquet Theory

In [ ]:
H_red = A + C
H_blue = B + D
ω = 1.0 * 2*π
args = {'w': ω}

# $\text{Full GSP}$

In [ ]:
# define cutoff
cutoff = 4

# define operators for bosonic modes 
a = destroy(cutoff)
Ib = identity(cutoff)

# define local Hamiltonians
A = H12 = H4_list[0]
B = H23 = H4_list[1]
C = H34 = H4_list[2]
D = H41 = H4_list[3]

In [ ]:
# define operators in argument of matrix exponentials 
α = 0.1
α_arg = α*a.dag() - np.conj(α)*a

# can also use *tuple([Ib]*N) for repeated Ib's
α1 = tensor([α_arg, Ib, Ib, Ib])
α2 = tensor([Ib, α_arg, Ib, Ib])
α3 = tensor([Ib, Ib, α_arg, Ib])
α4 = tensor([Ib, Ib, Ib, α_arg])

In [ ]:
# display method of exponentiating bosonic matrices 
# that contain tensor products of identities
display(α1.expm() == (tensor(α_arg, Ib, Ib, Ib)).expm() == tensor(α_arg.expm(), Ib, Ib, Ib))
display(α2.expm() == (tensor(Ib, α_arg, Ib, Ib)).expm() == tensor(Ib, α_arg.expm(), Ib, Ib))
display(α3.expm() == (tensor(Ib, Ib, α_arg, Ib)).expm() == tensor(Ib, Ib, α_arg.expm(), Ib))
display(α4.expm() == (tensor(Ib, Ib, Ib, α_arg)).expm() == tensor(Ib, Ib, Ib, α_arg.expm()))

In [ ]:
# display how you can factor bosonic matrix exponentials 
# into products of matrix exponentials
display((α1 + α2).expm() == tensor(α_arg, Ib, Ib, Ib).expm()*tensor(Ib, α_arg, Ib, Ib).expm() == \
                            tensor(α_arg.expm(), α_arg.expm(), Ib, Ib))
display((α2 + α3).expm() == tensor(Ib, α_arg, Ib, Ib).expm()*tensor(Ib, Ib, α_arg, Ib).expm() == \
                            tensor(Ib, α_arg.expm(), α_arg.expm(), Ib))
display((α3 + α4).expm() == tensor(Ib, Ib, α_arg, Ib).expm()*tensor(Ib, Ib, Ib, α_arg).expm() == \
                            tensor(Ib, Ib, α_arg.expm(), α_arg.expm()))
display((α4 + α1).expm() == tensor(Ib, Ib, Ib, α_arg).expm()*tensor(α_arg, Ib, Ib, Ib).expm() == \
                            tensor(α_arg.expm(), Ib, Ib, α_arg.expm()))

In [ ]:
# display method of exponentiating Hamiltonians that 
# involve tensor products of identities
display(A.expm() == H12.expm() == tensor(H2, I2, I2).expm() == tensor(H2.expm(), I2, I2))
display(B.expm() == H23.expm() == tensor(I2, H2, I2).expm() == tensor(I2, H2.expm(), I2))
display(C.expm() == H34.expm() == tensor(I2, I2, H2).expm() == tensor(I2, I2, H2.expm()))
display(D.expm() == H41.expm() == tensor(H2, I2, I2).permute([0, 3, 2, 1]).expm() == \
                                  tensor(H2.expm(), I2, I2).permute([0, 3, 2, 1]))

In [ ]:
# functions for displacement operators weighted by A
def D12_A(α, cutoff): 
    
    '''
    operator that displaces modes 1 & 2 by weight of A = H12
    '''
    
    # define bosonic operators 
    a = destroy(cutoff)
    Ib = qeye(cutoff)
    
    # define argument of unconditional displacement operator
    α_arg = α*a.dag() - np.conj(α)*a
    
    return (tensor(tensor(α_arg, H2).expm(), Ib, Ib, Ib, I2, I2)) \
            .permute([0, 5, 3, 4, 1, 2, 6, 7]) * \
           (tensor(Ib, tensor(α_arg, H2).expm(), Ib, Ib, I2, I2)) \
            .permute([0, 1, 5, 4, 2, 3, 6, 7])
def D34_A(α, cutoff):
    
    '''
    operator that displaces modes 3 & 4 by weight of A = H12
    '''
    
    # define bosonic operators 
    a = destroy(cutoff)
    Ib = qeye(cutoff)
    
    # define argument of unconditional displacement operator
    α_arg = α*a.dag() - np.conj(α)*a
    
    return (tensor(Ib, Ib, tensor(α_arg, H2).expm(), Ib, I2, I2)) \
            .permute([0, 1, 2, 5, 3, 4, 6, 7]) * \
           (tensor(Ib, Ib, Ib, tensor(α_arg, H2).expm(), I2, I2)) 

# functions for displacement operators weighted by B
def D23_B(α, cutoff): 
    
    '''
    operator that displaces modes 2 & 3 by weight of B = H23
    '''
    
    # define bosonic operators 
    a = destroy(cutoff)
    Ib = qeye(cutoff)
    
    # define argument of unconditional displacement operator
    α_arg = α*a.dag() - np.conj(α)*a
    
    return (tensor(Ib, tensor(α_arg, H2).expm(), Ib, I2, Ib, I2)) \
            .permute([0, 1, 6, 4, 5, 2, 3, 7]) * \
           (tensor(Ib, Ib, tensor(α_arg, H2).expm(), I2, Ib, I2)) \
            .permute([0, 1, 2, 6, 5, 3, 4, 7])
def D41_B(α, cutoff):
    
    '''
    operator that displaces modes 4 & 1 by weight of B = H23
    '''
    
    # define bosonic operators 
    a = destroy(cutoff)
    Ib = qeye(cutoff)
    
    # define argument of unconditional displacement operator
    α_arg = α*a.dag() - np.conj(α)*a
    
    return (tensor(Ib, Ib, Ib, tensor(α_arg, H2).expm(), I2, I2)) \
            .permute([0, 1, 2, 3, 6, 4, 5, 7]) * \
           (tensor(tensor(α_arg, H2).expm(), Ib, Ib, I2, Ib, I2)) \
            .permute([0, 6, 3, 4, 5, 1, 2, 7])

# functions for displacement operators weighted by C
def D34_C(α, cutoff): 
    
    '''
    operator that displaces modes 3 & 4 by weight of C = H34
    '''
    
    # define bosonic operators 
    a = destroy(cutoff)
    Ib = qeye(cutoff)
    
    # define argument of unconditional displacement operator
    α_arg = α*a.dag() - np.conj(α)*a
    
    return (tensor(Ib, Ib, tensor(α_arg, H2).expm(), I2, I2, Ib)) \
            .permute([0, 1, 2, 7, 5, 6, 3, 4]) * \
           (tensor(Ib, Ib, Ib, tensor(α_arg, H2).expm(), I2, I2)) \
            .permute([0, 1, 2, 3, 7, 6, 4, 5])
def D12_C(α, cutoff):
    
    '''
    operator that displaces modes 1 & 2 by weight of C = H34
    '''
    
    # define bosonic operators 
    a = destroy(cutoff)
    Ib = qeye(cutoff)
    
    # define argument of unconditional displacement operator
    α_arg = α*a.dag() - np.conj(α)*a
    
    return (tensor(tensor(α_arg, H2).expm(), Ib, Ib, I2, I2, Ib)) \
            .permute([0, 7, 3, 4, 5, 6, 1, 2]) * \
           (tensor(Ib, tensor(α_arg, H2).expm(), Ib, I2, I2, Ib)) \
            .permute([0, 1, 7, 4, 5, 6, 2, 3])

# functions for displacement operators weighted by C
def D41_D(α, cutoff): 
    
    '''
    operator that displaces modes 4 & 1 by weight of D = H41
    '''
    
    # define bosonic operators 
    a = destroy(cutoff)
    Ib = qeye(cutoff)
    
    # define argument of unconditional displacement operator
    α_arg = α*a.dag() - np.conj(α)*a
    
    return (tensor(Ib, Ib, Ib, tensor(α_arg, H2).expm(), I2, I2)) \
            .permute([0, 1, 2, 3, 4, 7, 6, 5]) * \
           (tensor(tensor(α_arg, H2).expm(), Ib, Ib, Ib, I2, I2)) \
            .permute([0, 3, 4, 5, 1, 6, 7, 2])
def D23_D(α, cutoff):
    
    '''
    operator that displaces modes 2 & 3 by weight of D = H41
    '''
    
    # define bosonic operators 
    a = destroy(cutoff)
    Ib = qeye(cutoff)
    
    # define argument of unconditional displacement operator
    α_arg = α*a.dag() - np.conj(α)*a
    
    return (tensor(Ib, tensor(α_arg, H2).expm(), Ib, Ib, I2, I2)) \
            .permute([0, 1, 4, 5, 2, 6, 7, 3]) * \
           (tensor(Ib, Ib, tensor(α_arg, H2).expm(), Ib, I2, I2)) \
            .permute([0, 1, 2, 5, 3, 6, 7, 4])

In [ ]:
# define cutoff
cutoff = 4

# define operators for bosonic modes 
a = destroy(cutoff)
Ib = identity(cutoff)

# define operators in argument of matrix exponentials 
α = 0.1
α_arg = α*a.dag() - np.conj(α)*a

# can also use *tuple([Ib]*N) for repeated Ib's
α1 = tensor([α_arg, Ib, Ib, Ib])
α2 = tensor([Ib, α_arg, Ib, Ib])
α3 = tensor([Ib, Ib, α_arg, Ib])
α4 = tensor([Ib, Ib, Ib, α_arg])


# evaluate functions for displacements operators weighted by A
display(D12_A(α, cutoff) == (tensor(α1 + α2, A)).expm())
display(D34_A(α, cutoff) == (tensor(α3 + α4, A)).expm())

# evaluate functions for displacements operators weighted by B
display(D23_B(α, cutoff) == (tensor(α2 + α3, B)).expm())
display(D41_B(α, cutoff) == (tensor(α4 + α1, B)).expm())

# evaluate functions for displacements operators weighted by C
display(D34_C(α, cutoff) == (tensor(α3 + α4, C)).expm())
display(D12_C(α, cutoff) == (tensor(α1 + α2, C)).expm())

# evaluate functions for displacements operators weighted by D
display(D41_D(α, cutoff) == (tensor(α4 + α1, D)).expm())
display(D23_D(α, cutoff) == (tensor(α2 + α3, D)).expm())

In [ ]:
def GSP(α, cutoff): 
    
    '''
    return ground state projection operator for N = 4 sites
    '''
    
    return D23_D(α, cutoff)*D41_B(α, cutoff)*D12_C(α, cutoff)*D34_A(α, cutoff) * \
           D41_D(α, cutoff)*D23_B(α, cutoff)*D34_C(α, cutoff)*D12_A(α, cutoff)

In [ ]:
α_start = 0
α_end = 1
α_steps = 10
cutoff = 3
α_array = np.linspace(α_start, α_end, α_steps)

start_time = time.time()
GSP_list3 = [GSP(α, cutoff) for α in α_array]
print("%s seconds" % (time.time() - start_time))

In [ ]:
α_start = 0
α_end = 1
α_steps = 10
cutoff = 4
α_array = np.linspace(α_start, α_end, α_steps)

start_time = time.time()
GSP_list4 = [GSP(α, cutoff) for α in α_array]
print("%s seconds" % (time.time() - start_time))

In [ ]:
GSP_op = GSP(α, 5)

In [ ]:
19*13*6

In [ ]:
18*14*8